<a href="https://colab.research.google.com/github/markriedl/weirdai/blob/master/weird_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Weird A.I Yankovic

<img src="https://www.dropbox.com/s/h6xaopym08o977t/weird_ai_logo.JPG?dl=1" alt="Weird A.I. Yankovic logo" width="200px"/>

Weird A.I. Yankovic is a neural network based lyric generation system. Given a syllable and rhyme scheme, it attempts to generate new lyrics that fit that scheme.

The intended use is to generate new lyrics for existing songs by feeding in the syllable and rhyme scheme for the song and then some contextualization information.

It does not sing or match the lyrics to the music. You have to do that yourself. To make that easier, there are routines at the end for creating a karaoke video.

This system was developed by [Mark Riedl](https://eilab.gatech.edu/mark-riedl). The work was performed independently of the Georgia Institute of Technology and released under MIT License.  


# Code

## Install stuff

In [1]:
!apt-get install festival espeak-ng
!pip install phonemizer
!pip install transformers
!pip install pronouncing
!pip install wordfreq

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  alsa-utils espeak-ng-data festlex-cmu festlex-poslex festvox-kallpc16k
  libespeak-ng1 libestools2.5 libfftw3-single3 libnewt0.52 libpcaudio0
  libsonic0 sgml-base whiptail
Suggested packages:
  pidgin-festival festival-freebsoft-utils libfftw3-bin libfftw3-dev
  sgml-base-doc
The following NEW packages will be installed:
  alsa-utils espeak-ng espeak-ng-data festival festlex-cmu festlex-poslex
  festvox-kallpc16k libespeak-ng1 libestools2.5 libfftw3-single3 libnewt0.52
  libpcaudio0 libsonic0 sgml-base whiptail
0 upgraded, 15 newly installed, 0 to remove and 31 not upgraded.
Need to get 12.1 MB of archives.
After this operation, 37.4 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 sgml-base all 1.29 [12.3 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 libnewt0.52 amd64 0.5

In [0]:
import torch
import torch.nn
import torch.nn.functional as F
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import XLNetTokenizer, XLNetLMHeadModel
from transformers import BertModel, BertTokenizer
import pdb
import re
import os
import copy
from functools import reduce
import string
import random
import shutil
import functools
from phonemizer.phonemize import phonemize
from phonemizer.separator import Separator
import pronouncing
from wordfreq import word_frequency, top_n_list


## Set Up File Paths

This is going to work faster if you mount your Google Drive and create a directory.

In [3]:
### If you want to store rhyme dictionary and phone cache in your Google Drive
### First mount your drive and create a "weirdai" directory
### If false, will download the rhyme dictionary each time and recreate an empty phone cache each time
MY_DRIVE = '/content/drive/My Drive'
WEIRD_AI_PATH = os.path.join(MY_DRIVE, 'weirdai')
NEAR_RHYME_DICT_FILENAME = 'near_rhymes20000'
PHONE_CACHE_FILENAME = 'phone_cache'
NEAR_RHYME_PATH = os.path.join('/content/', NEAR_RHYME_DICT_FILENAME)
PHONE_CACHE_PATH = os.path.join('/content/', PHONE_CACHE_FILENAME)
if not os.path.exists(MY_DRIVE):
  print("Google Drive not mounted.")
elif not os.path.exists(WEIRD_AI_PATH):
  try:
    os.mkdir(WEIRD_AI_PATH)
    print("Making new directory", WEIRD_AI_PATH)
  except:
    print("Could not make cache directory in Google Drive.")

if os.path.exists(WEIRD_AI_PATH):
  NEAR_RHYME_PATH = os.path.join(WEIRD_AI_PATH, NEAR_RHYME_DICT_FILENAME)
  PHONE_CACHE_PATH = os.path.join(WEIRD_AI_PATH, PHONE_CACHE_FILENAME)
  print("Setting near rhyme dictionary path to", NEAR_RHYME_PATH)
  print("Setting phoneme cache to", PHONE_CACHE_PATH)


Setting near rhyme dictionary path to /content/drive/My Drive/weirdai/near_rhymes20000
Setting phoneme cache to /content/drive/My Drive/weirdai/phone_cache


## Global Parameters

Some global parameters you can play with.

In [0]:
### Top-k most frequent rhymes
RHYME_K = 50                          
# for top-k sampling of tokens from language models
SAMPLE_K = 40                         
# How many times to try a line
NUM_TRIES = 6                         
### Always pick the best scoring line?
GREEDY_PICK_LINE = False
### temperature for picking different possible options for lines. Set to 1 to be most permissive. 
### Set closer to zero to be more conservative and prefer the best scoring line more.
PICK_LINE_TEMPERATURE = 0.1
### Setting this too high can cause memory problems
MAX_CONTEXT_LENGTH = 125

## Rhyme dectection

Helper functions

In [0]:
### These are legit letters
STR_LETTERS = set(string.ascii_letters + string.digits)

### Is a string a word?
def is_word(str):
  str_set = set(str)
  return len(str) > 0 and str_set.issubset(STR_LETTERS)

### Is a letter a vowel?
def is_vowel(letter):
  return letter in ['a', 'e', 'i', 'o', 'u', 'y']

### Is a letter a consonant?
def is_consonant(letter):
  return not is_vowel(letter)

### Is a phoneme a vowel sound?
### Pass in a phoneme (get this from Phonetic.phones_list)
def is_vowel_sound(phoneme):
  return is_vowel(phoneme[0])# and (is_vowel(phoneme[-1]) or phoneme[-1] in ['h', 'x', 'y'])

### Is a phoneme a consonant sound?
### Pass in a phoneme (get this from Phonetic.phones_list)
def is_consonant_sound(phoneme):
  return not is_vowel_sound(phoneme)

### Return the number of vowel phonemes in a phoneneme list.
### Pass in a list of phonemes (get this from Phonetic.phones_list)
def num_vowel_phones(phone_list):
  count = 0
  for ph in phone_list:
    if is_vowel(ph[0]):
      count = count + 1
  return count

### For whatever reason, the first call to phonemize crashes because the API call fails.
### This calls phonemize() once with a burner word just to get it out of the system
def test_phonemize():
  word = 'foobar'
  try:
    festival = phonemize(word, separator=Separator(phone='.', syllable='|', word=' ')).strip()
  except:
    pass
  try:
    espeak = phonemize(word, backend='espeak', with_stress=True, separator=Separator(phone='.', syllable='', word=' ')).strip()
  except:
    pass

### Return the indicies of a character (ch) in a string (s)
def find_char_indexes(s, ch):
    return [i for i, ltr in enumerate(s) if ltr == ch]

### Gets the phonemize API going
test_phonemize()

Get Phonemes and syllables for words.

A Phonetic object stores several phonetic variations for a given word and knows the number of syllables for the word.

The Phonetic constructor is given the word.

Getting phonetic information is slow, so it caches its results in ``PHONE_CACHE``, which should be saved to disk to speed up processing.

In [0]:
### Store phoneme information here for fast lookup
PHONE_CACHE = {}  # Store results in cache for faster lookup

### Phonetic class
### word: orignal word
class Phonetic():
  def __init__(self, word):
    global PHONE_CACHE
    self.word = word                  # Remember the word
    festival = None                   # festival results
    espeak = None                     # espeak results
    # If the word is in cache, use those results
    if word in PHONE_CACHE:
      festival, espeak = PHONE_CACHE[word]
    else:
      # API calls
      festival = phonemize(word, separator=Separator(phone='.', syllable='|', word=' ')).strip()
      espeak = phonemize(word, backend='espeak', with_stress=True, separator=Separator(phone='.', syllable='', word=' ')).strip()
      # Store the results
      PHONE_CACHE[word] = (festival, espeak)
    # List of syllables. Each syllable is a list of phones
    self.syllables_with_phones = [syl[:-1].split('.') for syl in festival.split('|')[:-1]]
    # Just the syllables in raw form
    self.syllables = festival.replace('.', '')
    # Just the syllables in list form
    self.syllables_list = self.syllables.split('|')[:-1]
    # Just the phones in raw form
    self.phones = festival.replace('|', '')
    # Just the phones in list form
    self.phones_list = self.phones.split('.')[:-1]
    self.major_stress = 0       # The major stresses phone (index)
    self.minor_stresses = []    # List of minor stressed phones (list of indices)
    # Compute the major stress
    major_stress_char_idxs = find_char_indexes(espeak, 'ˈ')
    if len(major_stress_char_idxs) > 0:
      stress_idx = espeak[:major_stress_char_idxs[0]].count('.')
      if stress_idx < len(self.phones_list):
        self.major_stress = stress_idx
      else:
        # Find the next earliest vowel phone
        for i in range(len(self.phones_list)):
          if is_vowel_sound(self.phones_list[len(self.phones_list)-i-1]):
            self.major_stress = len(self.phones_list) - i - 1
            break
    # compute the minor stresses      
    minor_stress_char_idxs = find_char_indexes(espeak, 'ˌ')
    if len(minor_stress_char_idxs) > 0:
      for char_idx in minor_stress_char_idxs:
        stress_idx = espeak[:char_idx].count('.')
        if stress_idx < len(self.phones_list):
          self.minor_stresses.append(stress_idx)
        else:
          # Find the next earliest vowel phone
          for i in range(len(self.phones_list)):
            if is_vowel_sound(self.phones_list[len(self.phones_list)-i-1]):
              self.minor_stresses.append(len(self.phones_list)-i-1)
              break
      # all stressed phones (list of indices)
    self.all_stresses = sorted([self.major_stress] + self.minor_stresses[:])

  def num_phones(self):
    return len(self.phones_list)

  def num_syllables(self):
    return len(self.syllables_list)

  def get_num_vowels(self):
    return num_vowel_phones(self.phones_list)

  def get_nth_vowel_phone(self, n=0):
    count = -1
    for i, ph in enumerate(self.phones_list):
      if is_vowel_sound(ph):
        count = count + 1
      if count == n:
        return ph, i
    else:
      return None

  def get_syllable_of_nth_phone(self, n):
    count = -1
    for i, syl in enumerate(self.syllables_list):
      for ph in syl:
        count = count + 1
        if count == n:
          return i
    return None

### Save the phone cache to disk
def save_phone_cache(cache, filename):
  with open(filename, 'w') as f:
    for key in list(cache.keys()):
      festival, espeak = cache[key]
      f.write(key + '\t' + festival + '\t' + espeak + '\n')

### Load the phone cache from disk
def load_phone_cache(filename):
  cache = {}
  for line in open(filename, 'r'):
    line = line.strip()
    split_line = line.split('\t')
    if len(split_line) >= 3:
      word, festival, espeak = split_line
      cache[word] = (festival, espeak)
  return cache

def load_near_rhyme_dictionary(filename):
  rhyme_dict = {}
  for line in open(filename, 'r'):
    line = line.split('\t')
    key = line[0].strip()
    val = line[1].strip()
    if key not in rhyme_dict:
      rhyme_dict[key] = []
    rhyme_dict[key].append(val)
  return rhyme_dict

Detect perfect rhyme

In [0]:
### Return true if two words are perfect ryles of each other
def perfect_rhyme(word1, word2):
  phonetic1 = Phonetic(word1)
  phonetic2 = Phonetic(word2)
  stress1 = phonetic1.all_stresses[-1]
  stress2 = phonetic2.all_stresses[-1]
  if phonetic1.num_phones() - stress1 != phonetic2.num_phones() - stress2:
    return False
  for i in range(phonetic1.num_phones() - stress1):
    phone1 = phonetic1.phones_list[i + stress1]
    phone2 = phonetic2.phones_list[i + stress2]
    if phone1 != phone2:
      return False
  return True

Detect near rhyme at the phenome level

In [0]:
### Let's pretend that these phones are the same
### (In addition all vowel phones that start with the same letter will be considered the same)
NEAR_SETS = [['er', 'r'], 
             #['ih', 'eh'],
             ['eh', 'ah'],
             ['er', 'ax'],
             ['ae', 'ey'],
             ['t', 'f'],
             ['b', 'k'],
             ['p', 'z', 'th'],
             ['s', 'st', 'sk'],
             ['nt', 'ns'],
             ['n', 'ng']
             ]

### These phonemes start with the same letter but should not be considered eqivalent
FAR_SETS =[['aw', 'ay', 'ax'],
           ['ax', 'ao']]

### Determine if phonemes are a near match
def near_match(phone1, phone2):
  # Safety check
  if (phone1 is None) or (phone2 is None) or (len(phone1) == 0 and len(phone2) > 0) or (len(phone2) == 0 and len(phone1) > 0):
    return False
  if phone1 == phone2:
    # Phones are the same
    return True
  elif is_vowel_sound(phone1) and is_vowel_sound(phone2) and phone1[0] == phone2[0]:
    # Vowel sounds start with the same letter
    # Check that they are not in the same far set
    for fs in FAR_SETS:
      if phone1 in fs and phone2 in fs:
        return False
    return True
  else:
    # Check to see if the pair of phones are in the same near_set
    for ns in NEAR_SETS:
      if phone1 in ns and phone2 in ns:
        return True
  return False

### Check if every phone in list 1 is a near match to corresponding phone in list 2
def near_matches(phone_list1, phone_list2):
  if len(phone_list1) != len(phone_list2):
    return False
  else:
    for i in range(len(phone_list1)):
      if not near_match(phone_list1[i], phone_list2[i]):
        return False
  return True

Detect near rhyme at the word level

In [0]:
### If you turn this on, it will print information about the phoneme analysis being conducted
### every time near_rhyme() is called. Not recommended
VERBOSE = False

### Verbose print allows for debug printing to be turned off
def vprint(*args):
  if VERBOSE:
    print(' '.join([str(a) for a in args]))

### Determine if word1 and word2 are near rhymes
### if last_consonant is False, we won't check for near matches of the last consonant
def near_rhyme(word1, word2, last_consonant = True):
  if perfect_rhyme(word1, word2):
    return True
  phonetic1 = Phonetic(word1)
  phonetic2 = Phonetic(word2)
  stress1 = phonetic1.all_stresses[-1] # The last stressed phone
  stress2 = phonetic2.all_stresses[-1] # The last stressed phone
  vprint(phonetic1.syllables_with_phones, phonetic1.all_stresses)
  vprint(phonetic2.syllables_with_phones, phonetic2.all_stresses)
  
  # Word = a v x w c
  # a = prefix (not important)
  # v = last stressed vowel
  # x = any number of phones between v and w
  # w = last vowel
  # c = consonant after last vowel
  
  # Last stressed vowel (v)
  v1_index = stress1
  v2_index = stress2
  v1 = phonetic1.phones_list[v1_index]
  v2 = phonetic2.phones_list[v2_index]
  vprint('v:', v1, v1_index, v2, v2_index)

  # Last vowel (w)
  w1, w1_index = phonetic1.get_nth_vowel_phone(phonetic1.get_num_vowels()-1)
  w2, w2_index = phonetic2.get_nth_vowel_phone(phonetic2.get_num_vowels()-1)
  vprint('w:', w1, w1_index, w2, w2_index)

  # Consonants after last vowel (c)
  c1_index = w1_index + 1
  c1 = ''.join(phonetic1.phones_list[c1_index:phonetic1.num_phones()])
  c2_index = w2_index + 1
  c2 = ''.join(phonetic2.phones_list[c2_index:phonetic2.num_phones()])
  vprint('c:', c1, c1_index, c2, c2_index)

  # phones between v and w (x)
  x1 = phonetic1.phones_list[v1_index+1:w1_index]
  x2 = phonetic2.phones_list[v2_index+1:w2_index]
  vprint('x:', x1, x2)

  # p = first phone in x
  # q = last phone in x
  p1 = None
  q1 = None
  p2 = None
  q2 = None
  if len(x1) > 0:
    p1 = x1[0]
    q1 = x1[-1]
  if len(x2) > 0:
    p2 = x2[0]
    q2 = x2[-1]
  vprint('p,q:', p1, q1, p2, q2)

  if not near_match(w1, w2):
    vprint('w fail')
    return False
  elif not near_match(v1, v2):
    vprint('v fail')
    return False
  elif False and len(c1) != len(c2) and (len(c1) > 1 or len(c2) > 1): 
    vprint('c not same length - fail')
    return False
  elif last_consonant and not near_match(c1, c2):
    vprint('cs dont match - fail')
    return False
  elif len(x1) == 0 and len(x2) == 0:
    vprint("no x - match")
    return True
  elif len(x1) == 1 and len(x2) == 1 and near_match(x1[0], x2[0]):
    vprint('single x - match')
    return True
  elif len(x1) > 0 and len(x2) > 0 and num_vowel_phones(x1) != num_vowel_phones(x2):
    vprint('num vowel phones in x - fail')
    return False
  elif near_match(p1, p2) and near_match(q1, q2):
    vprint('ps or qs - match')
    return True
  return False


## Load Near Rhyme Dictionary

Download the near-rhyme dictionary

In [0]:
if not os.path.exists(NEAR_RHYME_PATH): 
  !wget https://www.dropbox.com/s/8a800ivlp0uknic/near_rhymes20000.zip?dl=1 -O near_rhymes20000.zip
  !unzip near_rhymes20000.zip
if not os.path.exists(PHONE_CACHE_PATH):
  !wget https://www.dropbox.com/s/i57hvmnlint7wj3/phone_cache.zip?dl=1 -O phone_cache.zip
  !unzip phone_cache.zip
if not os.path.exists(NEAR_RHYME_PATH):
  shutil.copyfile(NEAR_RHYME_DICT_FILENAME, NEAR_RHYME_PATH)
if not os.path.exists(PHONE_CACHE_PATH):
  shutil.copyfile(PHONE_CACHE_FILENAME, PHONE_CACHE_PATH)

Load the near-rhyme dictionary into memory

In [0]:
if os.path.exists(NEAR_RHYME_PATH):
  NEAR_RHYME_DICT = load_near_rhyme_dictionary(NEAR_RHYME_PATH)

if os.path.exists(PHONE_CACHE_PATH):
  PHONE_CACHE = load_phone_cache(PHONE_CACHE_PATH)

## Make a near-rhyme dictionary

Searching for near-rhymes is expensive. Pre-compute near-rhymes and store them to disk.

This only needs to be run once and the near-rhyme dictionary is made available for download. The only reason to run this function would be if you made changes to ``near_rhyme()``.

In [0]:
### Make a dictionary remembering whether words are near rhymes to each other.
def make_near_rhyme_dictionary(top_n, filename, rhyme_dict=None):
  global VERBOSE
  VERBOSE = False   # Turn off debugging prints
  near_rhymes = []
  # Get top n most frequent words
  top = top_n_list('en', top_n, wordlist='best')
  for i, w1 in enumerate(top):
    for j, w2 in enumerate(top):
      if is_word(w1) and is_word(w2) and w1 != w2:
        if rhyme_dict is None or w1 not in rhyme_dict or w2 not in rhyme_dict[w1]: 
          if near_rhyme(w1, w2):
            print("MATCH", i, j, w1, w2,)
            near_rhymes.append((w1, w2))
  if rhyme_dict is not None:
    for key in list(rhyme_dict.keys()):
      for val in rhyme_dict[key]:
        near_rhymes.append((key, val))
        near_rhymes.append((val, key))
  with open(filename, 'w') as f:
    for w1, w2 in near_rhymes:
      f.write(w1 + '\t' + w2 + '\n')

Don't run this unless you want to rebuild the near-ryme dictionary.

In [0]:
build_near_rhyme_dictionary = False #@param {type:"boolean"}
if build_near_rhyme_dictionary:
  make_near_rhyme_dictionary(20000, NEAR_RHYME_PATH)
  save_phone_cache(PHONE_CACHE, PHONE_CACHE_PATH)

## Load neural language models

GPT-2 is probably the best generator we use, but it only goes forward. We only use GPT-2 when we are generating a line that has an unconstrained rhyme.

In [14]:
GPT_TOKENIZER = GPT2Tokenizer.from_pretrained("gpt2")
GPT = GPT2LMHeadModel.from_pretrained('gpt2')
GPT.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): Laye

XLNET can generate forward and backward and fill in the middle of a sentence (sort of, we have to force it to operate this way).

In [15]:
XLNET_TOKENIZER = XLNetTokenizer.from_pretrained('xlnet-large-cased')
XLNET = XLNetLMHeadModel.from_pretrained('xlnet-large-cased')
XLNET.eval()

XLNetLMHeadModel(
  (transformer): XLNetModel(
    (word_embedding): Embedding(32000, 1024)
    (layer): ModuleList(
      (0): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
          (layer_1): Linear(in_features=1024, out_features=4096, bias=True)
          (layer_2): Linear(in_features=4096, out_features=1024, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (1): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((1024,), eps=1e-12, el

I was running into memory issues, so we only keep one neural language model in GPU memory at a time. This sets up ```@use_gpt``` and ```@use_xlnet``` function decorators. Put this before any function that will use either GPT-2 or XLNET. For example:
```
@use_gpt
def my_cool_function(inputs):
  outputs = GPT(inputs)
  return outputs
```



In [0]:
### Are GPUs available?
CUDA_AVAILABLE = torch.cuda.is_available()

# Which model is in the GPU (string)
MODEL_IN_GPU = None        
# Which models are we using and what are their names?
MODEL_HASH = {'gpt': GPT, 'xlnet': XLNET}

def _prep_model(model_name):
  global MODEL_IN_GPU
  global MODELS_NOT_IN_GPU
  if MODEL_IN_GPU != model_name:
    # Unload the model in the gpu (if any)
    if MODEL_IN_GPU is not None:
      MODEL_HASH[MODEL_IN_GPU].to('cpu')
      MODEL_IN_GPU = None
    # Load the new model to gpu
    if CUDA_AVAILABLE:
      print("LOADING", model_name)
      MODEL_HASH[model_name].to('cuda')
      MODEL_IN_GPU = model_name

# For backward compatibility:
def prep_gpt():
  _prep_model('gpt')

def prep_xlnet():
  _prep_model('xlnet')

# Decorators!
def use_gpt(func):
  def wrapper(*args, **kwargs):
    _prep_model('gpt')
    return func(*args, **kwargs)
  return wrapper

def use_xlnet(func):
  def wrapper(*args, **kwargs):
    _prep_model('xlnet')
    return func(*args, **kwargs)
  return wrapper

In [0]:
### This does top-k and top-p sampling from a list of logits. Borrowed from original GPT-2 code.
def top_k_top_p_filtering(logits, top_k=0, top_p=0.0, filter_value=-float('Inf')):
    """ Filter a distribution of logits using top-k and/or nucleus (top-p) filtering
        Args:
            logits: logits distribution shape (vocabulary size)
            top_k >0: keep only top k tokens with highest probability (top-k filtering).
            top_p >0.0: keep the top tokens with cumulative probability >= top_p (nucleus filtering).
                Nucleus filtering is described in Holtzman et al. (http://arxiv.org/abs/1904.09751)
    """
    assert logits.dim() == 1  # batch size 1 for now - could be updated for more but the code would be less clear
    top_k = min(top_k, logits.size(-1))  # Safety check
    if top_k > 0:
        # Remove all tokens with a probability less than the last token of the top-k
        indices_to_remove = logits < torch.topk(logits, top_k)[0][..., -1, None]
        logits[indices_to_remove] = filter_value

    if top_p > 0.0:
        sorted_logits, sorted_indices = torch.sort(logits, descending=True)
        cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

        # Remove tokens with cumulative probability above the threshold
        sorted_indices_to_remove = cumulative_probs > top_p
        # Shift the indices to the right to keep also the first token above the threshold
        sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
        sorted_indices_to_remove[..., 0] = 0

        indices_to_remove = sorted_indices[sorted_indices_to_remove]
        logits[indices_to_remove] = filter_value
    return logits

##Helpers

In [0]:
### These are punctuation
PUNCTUATION = ['.', ',', '-', '?', '!', ':', '_', '$', '%', '&', '#', '@', '*', '(', ')', '+', '=', '[', ']', '{', '}']
### These are numbers
NUMBERS = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '0']
### Periods are important
PERIOD = '.'
BLANK = ' '

### Is this string punctuation?
def is_punctuation(s):
  return len(set(s).intersection(set(PUNCTUATION))) > 0

### Remove punctuation from string
def remove_punctuation(s):
  return ''.join(i for i in s if not i in PUNCTUATION) 

### How many syllables in this string?
def get_syllables_for_line(line):
  line = ''.join(list(filter(lambda c: c in STR_LETTERS or c == BLANK, line)))
  count = 0
  words = line.split()
  for word in words:
    word = word.strip()
    if len(word) > 0 and is_word(word):
      phonetic = Phonetic(word)
      count = count + phonetic.num_syllables()
  return count

### Remove the prefix from the string (s)
def remove_prefix(s, prefix):
    rest = s[len(prefix):] if s.startswith(prefix) else s
    return rest

### The system considers whether to terminate a line after the line is generated.
### So punctuation of a line could end up in the next line down.
def fix_final_lines_punctuation(lines):
  new_lines = []
  for i, line in enumerate(lines):
    if i > 0 and line[0] in PUNCTUATION:
      new_lines[-1] = new_lines[-1] + line[0]
      new_lines.append(line[1:].strip())
    else: 
      new_lines.append(line.strip())
  return new_lines

def fix_final_lines_capitalization(lines):
  new_lines = []
  for line in lines:
    new_line = line[0].upper() + line[1:].lower()
    new_lines.append(new_line)
  return new_lines

### Figure out how to put two lines (strings) together.
### If force_break is True, then put a sentence break (period) between the two.
### Otherwise, try to figure out if there should be a period between.
def merge_lines(l1, l2, force_break = False):
  if len(l1) == 0:
    return l2
  elif len(l2) == 0 and force_break:
    return l1 + PERIOD
  elif len(l2) == 0:
    return l1
  elif not is_punctuation(l1[-1]) and force_break:
    return l1 + PERIOD + BLANK + l2
  else:
    return l1 + BLANK + l2

### Figure out which xlnet tokens are numbers
def xlnet_number_tokens():
  nums = []
  for i in range(len(XLNET_TOKENIZER.get_vocab())):
    s = XLNET_TOKENIZER.decode(i)
    if len(s) > 0 and len(set(s).intersection(set(NUMBERS))) > 0: #s[0] in NUMBERS:
      nums.append((i, s))
  return nums

### Figure out which gpt tokens are numbers
def gpt_number_tokens():
  nums = []
  for i in range(len(GPT_TOKENIZER.get_vocab())):
    s = GPT_TOKENIZER.decode(i)
    if len(s) > 0 and len(set(s).intersection(set(NUMBERS))) > 0: #s[0] in NUMBERS:
      nums.append((i, s))
  return nums

### Find all the numbers in xlnet and gpt vocabularies
XLNET_NUMBER_TOKENS = list(map(lambda x: x[0], xlnet_number_tokens()))
GPT_NUMBER_TOKENS = list(map(lambda x: x[0], gpt_number_tokens()))

### Print lyrics 
def pretty_print(lines):
  print('---------')
  for line in lines:
    print(line)
  print('---------')

EXCLAIMS = ['oh!', 'ah!', 'yeah!']

## If we don't have enough syllables, add some ohs ahs and yeahs
def add_filler(line, count, start):
  prefix = line[0:start]
  suffix = line[start:]
  filler = []
  for i in range(count):
    filler.append(random.choice(EXCLAIMS))
  return prefix + BLANK + BLANK.join(filler) + BLANK + suffix

## We get weird punctuation. Remove it
def remove_extra_punctuation(line):
  idx = len(line)
  for i in range(1, len(line)):
    pos = len(line) - i
    if line[pos] in PUNCTUATION:
      idx = pos
    else:
      break
  return line[0:idx]

### Merge two dictionaries.
### The dictionaries should contain lists as values.
def merge_dicts(dict1, dict2):
  if dict1 is None or dict2 is None:
    return {}
  new_dict = copy.deepcopy(dict1)
  for key in list(dict2.keys()):
    val_list = dict2[key]
    if key not in new_dict:
      new_dict[key] = []
    new_dict[key] += val_list
  return new_dict

### Count how many times each word occurs in a line.
def word_counts(line):
  counts = {}
  new_line = ''.join(list(filter(lambda c: c not in PUNCTUATION, line)))
  for word in new_line.split():
    if word not in counts:
      counts[word] = 0
    counts[word] += 1
  return counts

### Get rid of any lines that uses a word more than once
def filter_lines(lines):
  return list(filter(lambda line: max(word_counts(line).values()) <= 1, lines))

### Make sure our lines don't get too long
def crop_line(line):
  enc = GPT_TOKENIZER.encode(line)
  if len(enc) > MAX_CONTEXT_LENGTH:
    enc = enc[len(enc)-MAX_CONTEXT_LENGTH:]
  dec = GPT_TOKENIZER.decode(enc)
  return dec


##Post-processing



Provide a specification hash where keys are line numbers and the values are ```(pre, post)``` such that ```pre``` is a string (or None) that should be prepended to the line and ```post``` is a string (or none) that should be appended to the line.

```Pre``` and ```post``` can include special commands in brackets to do complex post-processing. Currently the commans supported are:

- ```{repeat n}``` to repeat the last n syllables in the line (or "all").
- ```{frepeat n}``` to repeat the first n syllables in the line (or "all")
- ```{copy n}``` to copy the nth line and insert it directly after the line indicated by the key

In [0]:
def repeat_fn(line, all_lines, line_number, spec, args):
  target_syllables = args[0]
  ref_line = all_lines[line_number]
  ref_line = ''.join(c for c in ref_line if not c in PUNCTUATION) 
  if target_syllables == 'all':
    return ref_line, all_lines, line_number, spec
  else:
    ref_line_words = ref_line.split()
    syllable_count = 0
    picked_words = []
    for word in reversed(ref_line_words):
      num_syllables = get_syllables_for_line(word)
      syllable_count = syllable_count + num_syllables
      picked_words.append(word)
      if syllable_count >= target_syllables:
        break
    return BLANK.join(reversed(picked_words)), all_lines, line_number, spec

def frepeat_fn(line, all_lines, line_number, spec, args):
  target_syllables = args[0]
  ref_line = all_lines[line_number]
  ref_line = ''.join(c for c in ref_line if not c in PUNCTUATION) 
  if target_syllables == 'all':
    return ref_line, all_lines, line_number, spec
  else:
    ref_line_words = ref_line.split()
    syllable_count = 0
    picked_words = []
    for word in reversed(ref_line_words):
      num_syllables = get_syllables_for_line(word)
      syllable_count = syllable_count + num_syllables
      picked_words.append(word)
      if syllable_count >= target_syllables:
        break
    return BLANK.join(reversed(picked_words)), all_lines, line_number, spec

def copy_fn(line, all_lines, line_number, spec, args):
  new_spec = {}
  line_to_copy = args[0]
  copied_line = all_lines[line_to_copy]
  new_all_lines = all_lines[0:line_number+1] + [copied_line] + all_lines[line_number+1:]
  for key in list(spec.keys()):
    val = spec[key]
    if key > line_number:
      new_spec[key+1] = val
    else:
      new_spec[key] = val
  return '', new_all_lines, line_number, new_spec

PARSE_FUNCTIONS = {'repeat': repeat_fn, "copy": copy_fn, 'frepeat': frepeat_fn}

def parse(line, all_lines, line_number, spec):
  done = False
  while not done:
    match = re.search(r'\{([\w]+)[ ]*([\w, ]*)\}', line)
    if match is None:
      done = True
    else:
      func = match.groups()[0]
      args = eval(match.groups()[1])
      if not isinstance(args, tuple):
        args = tuple([args])
      pre = line[0:match.start()]
      mid, all_lines, line_number, spec = PARSE_FUNCTIONS[func](line, all_lines, line_number, spec, args)
      post = line[match.end():]
      line = pre + mid + post
  return line, all_lines, line_number, spec

### Add post-processing information to each line
def post_process_lines(lines, spec):
  new_lines = []
  i = 0
  while i < len(lines):
    line = lines[i]
    if i in spec:
      pre, post = spec[i]
      if pre is not None:
        pre, lines, i, spec = parse(pre, lines, i, spec) if pre is not None else ''
        line = lines[i]
      else:
        pre = ''
      if post is not None:
        post, lines, i, spec = parse(post, lines, i, spec) if post is not None else ''
        line = lines[i]
      else:
        post = ''
      new_lines.append(pre + line + post)
    else:
      new_lines.append(line)
    i = i +1
  return new_lines


## Picking Rhymes

In [0]:
 ### Pick the word from the words list that is most similar to the context according to BERT
 @use_gpt
 def pick_similar(context, words, history = []): 
  token_hash = {}
  for word in words:
    token_hash[tuple(GPT_TOKENIZER.encode(word))] = None
  #prep_gpt()
  context_tokens = GPT_TOKENIZER.encode(context) 
  prompt = torch.tensor([context_tokens]) # context put into the right shape
  prompt = prompt.to('cuda') if CUDA_AVAILABLE else prompt
  past = None 
  beams = []
  for i in range(10):
    current_beam = []
    for j in range(20):
      # Generate
      output, new_past = GPT(prompt, past=past)
      # Top k filter: there are k real numbers and the rest are -inf
      logits = output[0][0][:]
      #top_k_logits = top_k_top_p_filtering(output[0][0], top_k=SAMPLE_K)
      tokens = torch.multinomial(F.softmax(logits), 1)
      token = tokens[0]
      if token == 13:
        break
      else:
        current_beam.append((token, logits))
        prompt = torch.tensor([token]).unsqueeze(0)
        prompt = prompt.to('cuda')
        past = new_past
    beams.append(current_beam)
  # ASSERT: we have 10 beams of 20 tokens or less
  for key in list(token_hash.keys()):
    sum = 0
    for beam in beams:
      for i in range(len(beam)):
        for j, tok in enumerate(key):
          if i+j < len(beam):
            logits = beam[i+j][1]
            sum = sum + logits[tok]
        if len(key) > 1:
          sum = sum / len(key)
      #if len(beam) > 1:
      #  sum = sum / len(beam)
    if token_hash[key] is None:
      token_hash[key] = sum
    else:
      token_hash[key] = token_hash[key] + sum
  # ASSERT: Token_hash populated with good values
  vals = torch.stack(list(token_hash.values()))
  soft = F.softmax(vals)
  mask = soft > 0.0
  num_nonzero = torch.sum(mask.int())
  topk = torch.multinomial(soft, min(10, len(words), num_nonzero.item()), replacement=False).tolist()
  print("Picking from:", list(map(lambda x: words[x], topk)), "given history", history)
  final_pick = None
  for idx in topk:
    if words[idx] not in history:
      final_pick = words[idx]
      break
  if final_pick is None:
    # Uh oh we didn't find anything, probably because all the hits were in history
    sorted_words = sorted(words, key=lambda w: token_hash[tuple(GPT_TOKENIZER.encode(w))], reverse=True)
    print("Second attempt", sorted_words, "history", history)
    final_pick = None
    for w in sorted_words:
      if w not in history:
        final_pick = w
        break
    if final_pick is None:
      weights = list(map(lambda w: -token_hash[tuple(GPT_TOKENIZER.encode(w))], sorted_words))
      print("Third attempt", list(zip(sorted_words, weights)), "history", history)
      final_pick = random.choices(sorted_words, weights)[0]
  print("Picking", final_pick)
  return final_pick

In [0]:
NO_RHYME_WORDS = ['francoise', "l'enfant", 'un', 'vanhove', "suhud",
                  're', 'le', 'tao', 'mao', 'lao', 'petr', 'chas', 'rao', 'raby', 'rabey', 
                  'ia', 'co', 'uk', 'a', 'i']

### Remove some words from rhyme dictionary
def filter_rhyme(word):
  return (PERIOD not in word) and \
          ('-' not in word) and \
          (len(word) > 1 or word == 'a' or word == 'I' or word == 'i') and \
          (word.lower() not in NO_RHYME_WORDS)

### Uniformly pick amongst top k most frequent perfect rhymes
def pick_perfect_rhyme(word, context = None, history=[]):
  # Remove unwanted letters
  word = ''.join(list(filter(lambda c: c in STR_LETTERS, word)))
  # Get rhymes
  rhymes = pronouncing.rhymes(word)
  # Remove unwanted words
  rhymes = list(filter(lambda w: filter_rhyme(w), rhymes))
  # If there aren't any rhymes, return the current word
  if len(rhymes) == 0:
    return word
  else:
    # Make sure k is smaller than total number of rhymes available
    k = min(RHYME_K, len(rhymes))
    # Get word frequency of rhymes
    probs = list(map(lambda r: word_frequency(r, 'en'), rhymes))
    probs_tensor = torch.tensor(probs)
    # Get top k
    vals, idxs = torch.topk(probs_tensor, k)
    if context is not None and len(context) > 0:
      candidates = [rhymes[x] for x in idxs.tolist()]
      pick = pick_similar(context, candidates, history)
      return pick
    else:
      # Pick uniformly
      r = random.randint(0, k-1)
      return rhymes[idxs[r].item()]

### Pick best near rhyme from the near rhyme dictionary
def pick_near_rhyme(word, context = None, history = []):
  global NEAR_RHYME_DICT
  if NEAR_RHYME_DICT is None:
    NEAR_RHYME_DICT = load_near_rhyme_dictionary(NEAR_RHYME_PATH)
  # Remove unwanted letters
  word = ''.join(list(filter(lambda c: c in STR_LETTERS, word)))
  # Get rhymes
  near_rhymes = []
  #rhymes = []
  if word in NEAR_RHYME_DICT:
    near_rhymes = NEAR_RHYME_DICT[word]
  if len(near_rhymes) > 0:
    rhymes = near_rhymes
  #else:
  #  rhymes = pronouncing.rhymes(word)
  rhymes = list(set(near_rhymes + pronouncing.rhymes(word)))
  # Remove unwanted words
  rhymes = list(filter(lambda w: filter_rhyme(w), rhymes))
  # If there aren't any rhymes, return the current word
  if len(rhymes) == 0:
    return word
  else:
    # Make sure k is smaller than total number of rhymes available
    k = min(RHYME_K, len(rhymes))
    # Get word frequency of rhymes
    probs = list(map(lambda r: word_frequency(r, 'en'), rhymes))
    probs_tensor = torch.tensor(probs)
    # Get top k
    vals, idxs = torch.topk(probs_tensor, k)
    if context is not None and len(context) > 0:
      candidates = [rhymes[x] for x in idxs.tolist()]
      pick = pick_similar(context, candidates, history)
      return pick
    else:
      # Pick uniformly
      r = random.randint(0, k-1)
      return rhymes[idxs[r].item()]

### Pick rhyme, redirects to pick_perfect_rhyme or pick_near_rhyme
def pick_rhyme(word, perfect=True, context=None, history=[]):
  if perfect:
    return pick_perfect_rhyme(word, context=context, history=history)
  else:
    return pick_near_rhyme(word, context=context, history=history)


## Lyrics Scoring

Compute a score for a segment of lyrics

In [0]:
LOSS = torch.nn.CrossEntropyLoss(reduction='sum')  

def interactive_pick_line(lines):
  result = None
  print("GENERATED CANDIDATES:")
  for i, line in enumerate(lines):
    print('[' + str(i) + '] ' + line)
  inp = input("CHOOSE BY NUMBER (0-" + str(len(lines)-1) + ") OR WRITE YOUR OWN: ")
  try:
    idx = int(inp)
    result = lines[idx]
  except ValueError:
    result = inp
  return result, INF


### Score a sentence, lower is better (cross entropy)
@use_gpt
def score_sentence(sentence):
  #encode sentence
  prompt = GPT_TOKENIZER.encode(sentence)
  # Set up x and y as shifted input
  x = torch.tensor([prompt[:-1]])
  y = torch.tensor(prompt[1:])
  if CUDA_AVAILABLE:
    x = x.to('cuda')
    y = y.to('cuda')
  #prep_gpt()  
  # Measure the logits for loss
  output, new_past = GPT(x, past=None)
  score = LOSS(output[0], y)
  return score


### Pick the best line. Run them all through the scoring function and pick the smallest
def pick_best_line(lines, context, history = [], interactive=False):
  if interactive:
    return interactive_pick_line(lines)
  else:
    # Remove duplicates
    modified_history = list(map(lambda s: remove_punctuation(s).encode('ascii', 'ignore').lower(), history))
    lines = list(filter(lambda s: remove_punctuation(s).encode('ascii', 'ignore').lower() not in modified_history, lines))
    # Score sentences
    scores = list(map(lambda l:score_sentence(merge_lines(context, l)), lines))
    scores_tensor = torch.tensor(scores, dtype=torch.float)
    #for i, line in enumerate(lines):
    #  print("SCORE", scores[i].item(), line)
    # Get smallest
    if len(lines) == 1:
      return lines[0], scores[0].item()
    elif GREEDY_PICK_LINE:
      vals, idxs = torch.topk(scores_tensor, 1, largest=False)
      idx = idxs[0].item()
      return lines[idx], vals[0].item()
    else:
      try:
        shifted = (scores_tensor-min(scores_tensor))
        flipped = (max(shifted) - shifted).div(max(shifted))
        exped = flipped.div(PICK_LINE_TEMPERATURE).exp()
        idxs = torch.multinomial(exped, 1)
        idx = idxs[0].item()
      except:
        # Probably a div by zero
        # Usually caused when all the sentences are the same and thus have the same scores
        # Which means shifted is a tensor of 0s
        idx = 0
      print("PICK", lines[idx])
      return lines[idx], scores[idx].item()

## Text Generation

Fill in masks between context and a rhyme (ending)

In [0]:
MASK_IDX = 6                          # MASK is token id 6
# Things I don't want generated
XLNET_NO_TOKENS = [6, 7, 8, 0, 1, 2, 3, 4, 5,
                   10, 11, 12, 13, 14, 15, 16, 2055, 6490, 26,
                   97, 167, 225, 4145, 3158, 17115, 22891, 17666, 4538] + XLNET_NUMBER_TOKENS
NINF = -float('Inf')                  # Negative infinity
INF = float('Inf')                    # Positive infinity

### Given a prompt, with one or more '<mask>' in it, fill the masks in with XLNET 
### Can go forward or backward. Backward seems to work better?
@use_xlnet
def fill_line(prompt, backward=False):
  generated_tokens = []
  # Convert prompt into tokens
  input_ids = torch.tensor(XLNET_TOKENIZER.encode(prompt, add_special_tokens=False)).unsqueeze(0)  
  # mask out the places we want to predict
  perm_mask = torch.zeros((1, input_ids.shape[1], input_ids.shape[1]), dtype=torch.float)
  masked = input_ids == MASK_IDX
  perm_mask = perm_mask + masked
  # The places we want to predict are...
  predicts = torch.nonzero(masked[0]).tolist()
  if backward:
    predicts = list(reversed(predicts))
  # Set up a diagonal where we want to predict, dim=0 is batch, dim=1 is each prediction
  target_mapping = torch.zeros((1, len(predicts), input_ids.shape[1]), dtype=torch.float)  
  for n, p in enumerate(predicts):
    target_mapping[0][n][p] = 1.0
  #prep_xlnet()
  if CUDA_AVAILABLE:
    input_ids = input_ids.to('cuda')
    perm_mask = perm_mask.to('cuda')
    target_mapping = target_mapping.to('cuda')

  # Fill one mask at a time until there are no places to fill (predicts is empty)
  while len(predicts) > 0:
    # Predict everything, but ignore all but the first prediction
    outputs = XLNET(input_ids, perm_mask=perm_mask, target_mapping=target_mapping)
    next_token_logits = outputs[0]  # Output has shape [target_mapping.size(0), target_mapping.size(1), config.vocab_size]
    # Filter to top-k
    logits = top_k_top_p_filtering(next_token_logits[0][0], top_k=SAMPLE_K)
    # Sample from top-k
    samples = torch.multinomial(F.softmax(logits), SAMPLE_K)
    # Make sure we didn't predict a repetition and not in NO_TOKENS
    pos = torch.nonzero(target_mapping[0][0]).item()
    previous_token = input_ids[0][pos-1].item()
    next_token = input_ids[0][pos+1].item()
    token = None
    for i in range(SAMPLE_K):
      tok = samples[i].item()
      if tok != previous_token and tok != next_token and tok not in XLNET_NO_TOKENS:
        # Avoid ALL punctuation
        if not is_punctuation(XLNET_TOKENIZER.decode(tok)):
          # avoid repeat tokens
          word = XLNET_TOKENIZER.decode(tok)
          if word.lower() not in XLNET_TOKENIZER.decode(generated_tokens).lower():
            token = tok
            generated_tokens.append(tok)
            break
    if token is None:
      token = samples[0].item()
      generated_tokens.append(token)
    # insert the token into the input
    input_ids[0][pos] = token
    #print(XLNET_TOKENIZER.decode(input_ids[0]))
    # Mask out everything that needs to be masked
    perm_mask = torch.zeros((1, input_ids.shape[1], input_ids.shape[1]), dtype=torch.float)
    perm_mask = perm_mask.to('cuda')
    masked = input_ids == 6
    perm_mask = perm_mask + masked
    # Update predicts, should be one less
    predicts = torch.nonzero(masked[0]).tolist()
    if backward:
      predicts = list(reversed(predicts))
    # Set up a diagonal where we want to predict, dim=0 is batch, dim=1 is each prediction
    target_mapping = torch.zeros((1, len(predicts), input_ids.shape[1]), dtype=torch.float)  
    for n, p in enumerate(predicts):
      target_mapping[0][n][p] = 1.0
    target_mapping = target_mapping.to('cuda')
  return XLNET_TOKENIZER.decode(input_ids[0])


Generate a line the ends in a rhyme (XLNET)

In [0]:
def generate_rhyme_line(rhyme, context, target_syllables, terminate_line=True):
  good_tries = [] # runs with the exact number of syllables
  bad_tries = {}  # Runs with fewer syllables (dict is num_syllables: list of lines)
  # i is number of masks
  # j is number of tries
  for i in range(int(target_syllables*1.5)):
    for j in range(NUM_TRIES):
      ## Even tries have a period added at the end of the previous context
      #if j % 2 == 0 and len(context) > 0 and context[-1] not in PUNCTUATION and not no_terminate_previous:
      #  context_copy = context[:] + PERIOD
      #else:
      context_copy = context[:] + BLANK
      # This fixes things up
      context_copy = XLNET_TOKENIZER.decode(XLNET_TOKENIZER.encode(context_copy, add_special_tokens=False))
      # Make masks
      prompt = context_copy + BLANK + BLANK.join(['<mask>']*(i+1)) + BLANK + rhyme
      if terminate_line or j % 2 == 1:
        prompt = prompt + PERIOD
      # Fill masks
      filled_line = fill_line(prompt, backward=True)
      # Figure out how many syllables we added
      candidate = remove_prefix(filled_line, context_copy).strip() # the newly added line
      line_syllable_count = get_syllables_for_line(candidate) # Number of syllables in newly added line
      # Did we get a good run?
      if line_syllable_count == target_syllables:
        good_tries.append(candidate)
        print("CANDIDATE", candidate)
      elif line_syllable_count < target_syllables:
        # Bad tries are those that are too short. Store in dict by length
        if line_syllable_count not in bad_tries:
          bad_tries[line_syllable_count] = []
        bad_tries[line_syllable_count].append(candidate)
  return good_tries, bad_tries

Generate an open-ended line (GPT)

In [0]:
GPT_NO_TOKENS = [59, 50256, 1, 6, 7, 8, 12, 14, 26, 58, 59, 60, 62, 90, 92, 2503, 3,
                 1906, 14988, 26391, 4023, 338] + GPT_NUMBER_TOKENS

@use_gpt
def generate_non_rhyme_line(context, target_syllables):
  generated_tokens = [] # Tokens generated along the way
  good_tries = [] # results with the correct number of syllables
  past_syllables = get_syllables_for_line(context) # How many syllables in the context
  #prep_gpt()
  # j is number of tries
  for j in range(NUM_TRIES):
    new_syllables = 0 # How many new syllables were produced
    # Odd tries add a period to the context
    #if j % 2 == 0 and len(context) and context[-1] not in PUNCTUATION and not no_terminate_previous:
    #  context_copy = context[:] + PERIOD
    #else:
    context_copy = context[:] + BLANK
    # Encode the context
    generated = GPT_TOKENIZER.encode(context_copy) # Used to collect up tokens
    prompt = torch.tensor([generated]) # context put into the right shape
    prompt = prompt.to('cuda') if CUDA_AVAILABLE else prompt
    past = None # Initially we don't have any history
    # Generate until we get enough syllables
    previous_token = generated[-1] if len(generated) > 0 else None
    count = 0
    while new_syllables < target_syllables and count < 1000: # break loop if too many iterations
      # Generate
      output, new_past = GPT(prompt, past=past)
      # Top k filter: there are k real numbers and the rest are -inf
      logits = top_k_top_p_filtering(output[0][0], top_k=SAMPLE_K)
      tokens = torch.multinomial(F.softmax(logits), SAMPLE_K)
      # Pick the first one from the top k that doesn't produce too many syllables
      for tok in tokens.tolist():
        # How many syllables do we have total?
        line_syllables = get_syllables_for_line(GPT_TOKENIZER.decode(generated + [tok]))
        # Have we gone over? Or generated a NO_TOKEN?
        if line_syllables <= target_syllables + past_syllables and tok not in GPT_NO_TOKENS:
          # We are good
          word = GPT_TOKENIZER.decode(tok) # The new word
          # Don't allow ANY punctuation
          if not is_punctuation(word):
            if word.lower() not in GPT_TOKENIZER.decode(generated_tokens).lower():
              generated_tokens.append(tok)
              # Add new word to generated
              generated.append(tok) 
              # Prep for the next run
              prompt = torch.tensor([tok]).unsqueeze(0)
              prompt = prompt.to('cuda')
              past = new_past
              new_syllables = line_syllables - past_syllables
              break
      count = count + 1
    candidate = remove_prefix(GPT_TOKENIZER.decode(generated), context).strip()
    print("CANDIDATE", candidate)
    good_tries.append(candidate)
  return good_tries, None

Generate a line that is guaranteed to end in a period (using XLNET).

In [0]:
def generate_terminal_non_rhyme_line(context, target_syllables):
  good_tries = [] # runs with the exact number of syllables
  bad_tries = {}  # Runs with fewer syllables (dict is num_syllables: list of lines)
  # i is number of masks
  # j is number of tries
  for i in range(int(target_syllables*1.5)):
    for j in range(NUM_TRIES):
      # Even tries have a period added at the end of the previous context
      #if j % 2 == 0 and len(context) > 0 and context[-1] not in PUNCTUATION and not no_terminate_previous:
      #  context_copy = context[:] + PERIOD
      #else:
      context_copy = context[:] + BLANK
      # This fixes things up
      context_copy = XLNET_TOKENIZER.decode(XLNET_TOKENIZER.encode(context_copy, add_special_tokens=False))
      # Make masks
      prompt = context_copy + BLANK + BLANK.join(['<mask>']*(i+1)) + PERIOD # Different from generate_rhyme_line
      # Fill masks
      filled_line = fill_line(prompt, backward=False) # Forward instead of backward
      # Figure out how many syllables we added
      candidate = remove_prefix(filled_line, context_copy).strip() # the newly added line
      line_syllable_count = get_syllables_for_line(candidate) # Number of syllables in newly added line
      # Did we get a good run?
      if line_syllable_count == target_syllables:
        good_tries.append(candidate)
        print("CANDIDATE", candidate)
      elif line_syllable_count < target_syllables:
        # Bad tries are those that are too short. Store in dict by length
        if line_syllable_count not in bad_tries:
          bad_tries[line_syllable_count] = []
        bad_tries[line_syllable_count].append(candidate)
  return good_tries, bad_tries

##Main execution loop

In [0]:
def run(context, scheme, rhyme_dict, use_near_rhymes = False, post = None, 
        recontextualize = False, interactive = False):
  ### SETUP #####################################
  lines = []    # Store the lyrics lines
  segments = [] # Store each segment separately
  # Set up the context
  context = context[:].strip()
  original_context = context[:]
  # Set up rhyme history
  rhyme_history = [] # The history of words used for rhymes
  # original context shouldn't have punctuation at the end, but the first prompt to the neural net should
  if len(context) >0 and context[-1] not in PUNCTUATION:
    context = context + PERIOD
  if len(original_context) > 0 and original_context[-1] in PUNCTUATION:
    original_context = original_context[:-1]
  # copy the rhyme_dict because we will be adding to it
  rhyme_dict = copy.deepcopy(rhyme_dict)
  ### ITERATE THROUGH SCHEME #####################
  # An entry can be tuples or lists of tuples. 
  # Easiest thing is to nest tuples into lists and treat everything the same
  for n, entry in enumerate(scheme):
    ### RECONTEXTUALIZE ##########################
    # Insert the original context into the history/context at sentence breaks
    if n > 0 and recontextualize:
      last_period = context.rfind(PERIOD)
      last_contextualization = context.rfind(original_context)
      if last_period >= 0 and last_contextualization >= 0 and last_contextualization + len(original_context) - 1 != last_period - 1:
        context = merge_lines(merge_lines(context[:last_period+1], original_context, force_break=True), context[last_period+1:], force_break=True)
    ### ITERATE THROUGH LINE SPEC #################
    current_line = []   # The line currently being worked on. May be made of several segments
    # if line is a single segment, nest it
    if isinstance(entry, tuple):
      entry = [entry]
    # A line has 1 or more tuples of the form (target_syllables, rhyme_index)
    for segment_num, segment in enumerate(entry):
      is_end_segment = (segment_num == len(entry) - 1)
      target_syllables = segment[0]
      rhyme_idx = segment[1]
      cmd = None
      goods = []    # lines with the right number of syllables
      shorts = {}   # lines with too few syllables
      new_context = None # The complete lyrics after a new line is added
      new_syllables = 0 # The number of new syllables added to lyrics
      terminate_segment = False
      ### PARSE SPECIAL COMMANDS ##################
      # We have commands to parse
      if len(segment) > 2:
        cmd = segment[2]
        terminate_segment = (cmd == ':end')
      print("LINE", n, "SEGMENT", segment_num, '(end)' if is_end_segment else '', "TARGET SYLLABLES", target_syllables, "RHYME INDEX", rhyme_idx, "COMMAND", cmd)
      # Check if we are filling in a partial line (when there is a rhyme or when there is a given verbatim string)
      if isinstance(rhyme_idx, str) or rhyme_idx in rhyme_dict:
        ### USE RHYME INDEX ########################
        # We are filling a line
        end_targets = [] # The line should end in this word (or words)
        ### RHYME INDEX IS A STRING ####
        if isinstance(rhyme_idx, str):
          # Use a verbatim string
          end_targets = [rhyme_idx] # rhyme_idx is actually a string
          # If target_syllables < 0 then set it to the exact number of syllables in the rhyme_idx string
          if target_syllables < 0:
            target_syllables = get_syllables_for_line(rhyme_idx)
        ### PICK A RHYME WORD ####
        else:
          # Get some rhyming words
          # but first fix the dictionary if there is a single word instead of a list
          if not isinstance(rhyme_dict[rhyme_idx], list) and not isinstance(rhyme_dict[rhyme_idx], tuple):
            rhyme_dict[rhyme_idx] = [rhyme_dict[rhyme_idx]]
          # Now get some words
          if rhyme_idx not in rhyme_dict:
            pdb.set_trace()
          end_targets = [pick_rhyme(w, perfect=not use_near_rhymes, context=context, history=rhyme_history) for w in rhyme_dict[rhyme_idx]]
        # There could be more than one possible rhyme target
        ### GENERATE #####
        for end_target in end_targets:
          print("RHYME TARGET", end_target)
          # Check to see if the end_target already fills up all of our syllable length
          end_target_syllables = get_syllables_for_line(end_target)
          if end_target_syllables < target_syllables:
            # Generate new lines
            meet_target_syllables, too_short = generate_rhyme_line(end_target, context, target_syllables, terminate_line=terminate_segment)
            goods = goods + meet_target_syllables
            shorts = merge_dicts(shorts, too_short)
          else:
            # Just copy the end target because that is all the syllables we need
            goods.append(end_target)
      else:
        ### NON-RHYME SEGMENT ###############################
        # We are generating a new line unconstrained
        # Generate new line
        if terminate_segment:
          # We've determined this segment must be terminal.
          match_target_syllables1, shorts = generate_terminal_non_rhyme_line(context, target_syllables)
          goods = match_target_syllables1
        else:
          # Segment does not need to be terminal
          match_target_syllables1, _ = generate_non_rhyme_line(context, target_syllables)
          match_target_syllables2 =  []
          if is_end_segment:
            match_target_syllables2, shorts = generate_terminal_non_rhyme_line(context, target_syllables)
          goods = match_target_syllables1 + match_target_syllables2
      ### PICK BEST CANDIDATE ############################################
      results = []
      if len(goods) > 0:
        results = goods
      else:
        longest_key = max(list(shorts.keys()))
        results = shorts[longest_key]
      best_continuation, score = pick_best_line(results, context, history=segments, interactive=interactive)
      new_syllables = get_syllables_for_line(best_continuation)
      ### UPDATE RHYME DICTIONARY AND RHYME HISTORY ####################################
      # If we are using near rhymes, store the near rhyme as possible target for future lines
      if not isinstance(rhyme_idx, str) and rhyme_idx >=0:
        # Make a new entry in rhyme_dict
        if (rhyme_idx not in rhyme_dict) or (rhyme_idx in rhyme_dict and use_near_rhymes):
          continuation = ''.join(list(filter(lambda c: c in STR_LETTERS or c == BLANK, best_continuation)))
          split_continuation = continuation.split(BLANK)
          end_word = split_continuation[-1]
          if end_word[-1] in PUNCTUATION:
            end_word = end_word[:-1]
          if rhyme_idx not in rhyme_dict:
            rhyme_dict[rhyme_idx] = []
          rhyme_dict[rhyme_idx].append(end_word)
        # Add to rhyme history
        rhyme_history.append(end_word)
      ### DO SOME FIXING #################################################
      # Fill in if we don't have enough syllables
      if new_syllables < target_syllables:
        rest = add_filler(best_continuation, target_syllables - new_syllables, 0)
      ### SET UP CONTEXT FOR NEXT ITERATION ##########################
      context = merge_lines(context, best_continuation)
      print('CONTEXT', context)
      current_line.append(best_continuation)
      segments.append(best_continuation)
      # get ready for the next iteration
      context = crop_line(context)
      previous_is_end_segment = is_end_segment
    ### DONE WITH LINE #################################################
    # put the current line together
    lines.append(BLANK.join(fix_final_lines_punctuation(current_line)))
  ### DONE WITH SPEC ###################################################
  #lines = fix_final_lines_punctuation(lines)
  lines = post_process_lines(lines, post)
  lines = fix_final_lines_capitalization(lines)
  return lines

## GUI Code

Extract the syllable and rhyme scheme from given lyrics.

In [0]:
def extract_scheme(lines, use_near_rhymes = True):
  syllables = []
  ends = []
  rhyme_lines = {}
  # Compute syllables per line
  for line in lines:
    line = line.strip()
    num_syl = get_syllables_for_line(line)
    syllables.append(num_syl)
    ends.append(line[-1] == '.')
  # Figure out which lines rhyme, piecewise
  for i, line1 in enumerate(lines):
    line1 = line1.strip()
    words1 = line1.split()
    last1 = words1[-1]
    for j, line2 in enumerate(lines):
      if i != j:
        line2 = line2.strip()
        words2 = line2.split()
        last2 = words2[-1]
        if perfect_rhyme(last1, last2) or (use_near_rhymes and near_rhyme(last1, last2)):
          if i not in rhyme_lines:
            rhyme_lines[i] = []
          if j not in rhyme_lines:
            rhyme_lines[j] = []
          if j not in rhyme_lines[i]:
            rhyme_lines[i].append(j)
          if i not in rhyme_lines[j]:
            rhyme_lines[j].append(i)
  # Gather up all rhyming lines
  for l1 in list(rhyme_lines.keys()):
    for l2 in rhyme_lines[l1]:
      for l3 in list(rhyme_lines[l2]):
        if l3 not in rhyme_lines[l1]:
          rhyme_lines[l1].append(l3)
  # reduce to unique sets
  rhyme_sets = []
  for l in list(rhyme_lines.values()):
    rhyme_set = set(l)
    if rhyme_set not in rhyme_sets:
      rhyme_sets.append(rhyme_set) 
  # Assign rhyme indexes to lines
  rhyme_idxs = {}
  for n, s in enumerate(rhyme_sets):
    for l in s:
      rhyme_idxs[l] = n
  # Build schema
  schema = []
  for n in range(len(lines)):
    segment = None
    syl_num = syllables[n]
    rhyme_idx = rhyme_idxs[n] if n in rhyme_idxs else -1
    if ends[n]:
      segment = (syl_num, rhyme_idx, ':end')
    else:
      segment = (syl_num, rhyme_idx)
    schema.append(segment)
  return schema

Callback hooks from javascript/html.

In [0]:
import IPython
from google.colab import output

def python_run_hook(prompt, text, use_near_rhymes = False, recontextualize = False):
  text = text.strip()
  lines = text.split('\n')
  prompt = prompt.strip()
  scheme = None
  try:
    if text[0] == '[':
      scheme = eval(text)
    else:
      scheme = extract_scheme(lines, use_near_rhymes)
    print(scheme)
    lines = run(prompt, scheme, {}, recontextualize=recontextualize, use_near_rhymes=use_near_rhymes, post=[])
    pretty_print(lines)
    return IPython.display.JSON({'result': 'true'})
  except:
    return IPython.display.JSON({'result': 'false'})


output.register_callback('notebook.python_run_hook', python_run_hook)

# GUI

You can launch the system by entering 

- Context, a short phrase about the topic of the lyrics.
- The lyrics from an existing song with carriage returns at the end of each line. If a line has a period at the end, the system will honor it. Lines without periods may or may not get sentence breaks.

The system will reverse engineer the syllable pattern and the rhyme scheme (only looking for the last words of each line).

You can choose whether to use near rhymes (if unchecked, the system will only use perfect rhymes).

You an choose to recontextualize. The system will attempt to reintroduce the context phrase throughout to keep the lyrics on topic. No guarantees.

How to use:

1. From the menu at the top, select Runtime >> Run all
2. Scroll down below this cell to enter lyrics and context prompt.

In [30]:
%%html
<!DOCTYPE html>
<html>
<head>
<meta name="viewport" content="width=device-width, initial-scale=1.0"/>

<style>
// MAKE CANVAS 
canvas {
    border:1px solid #d3d3d3;
    background-color: #f1f1f1;
}
</style>
</head>
<body>
<script>
function js_run() {
  var the_text = "";
  var prompt = "";
  var recontextualize = false;
  var use_near_rhymes = false;
  var input_lyrics = document.getElementById("inp_lyrics");
  var input_prompt = document.getElementById("inp_prompt");
  var input_recontextualize = document.getElementById("inp_recontextualize");
  var input_near_rhymes = document.getElementById("inp_near_rhymes");
  if (input_lyrics.value.length == 0) {
    return;
  }
  if (input_prompt.value.length == 0) {
    return;
  }
  recontextualize = input_recontextualize.checked;
  use_near_rhymes = input_near_rhymes.checked;
  the_text = input_lyrics.value;
  prompt = input_prompt.value;
  // Call python
  (async function() {
    const result = await google.colab.kernel.invokeFunction(
      'notebook.python_run_hook', // The callback name.
      [prompt, the_text, use_near_rhymes, recontextualize], // The arguments.
      {}); // kwargs
    const res = result.data['application/json'];
    //document.querySelector("#output-area").appendChild(document.createTextNode(text.result));
  })();
}
</script>
<strong>Context prompt:</strong> <input id="inp_prompt" /><br>
<strong>Enter lyrics here:</strong><br />
<textarea rows = "5" cols = "60" name = "description" id="inp_lyrics">
</textarea><br />
<strong>Recontextualize?</strong> <input type="checkbox" id="inp_recontextualize" /><br />
<strong>Use near rymes?</strong> <input type="checkbox" id="inp_near_rhymes" /><br />
<button onmouseup="js_run()">Run!</button><br>
</body>
</html>

# Advanced User Mode

## Rhyme Scheme

A scheme indicates how many lines, how many syllables per line, and which lines rhyme with each other.

A scheme is a list of tuples where each tuple is the number of syllables in a segment and an index (number)
indicating which other segments to rhyme with.
For example ```[(5, 1), (6, 2), (7, 1), (8, 2)]``` would indicate four segments, where the first and third segments rhyme. The first segment has five syllables, the second segment has six syllables, and so on.

Instead of a rhyme index, you can also provide a word or phrase that the segment should end with.
The number of syllables for that segments should be greater than or equal to the number of syllables
in the word or phrase. For example, ```[(5, 1), (6, 1), (7, 'dead')]``` would make sure the last word of the last
segment ended in the word ```'dead'```. 
Likewise ```[(3, 'hello world'), (6, 1), (7, 1)]``` would start with the entire first 
segment being 'hello world'. 

For most situations, a segment would correspond to a complete line on lyrics. But in some cases you want multiple segments per line, so as to have rhymes within a line. In this case, you can wrap tuples in a list, such as ```[(6, 1), [(4, 2), (4, 2), (4, 1)], (7, 1)]```, which would have three lines consisting of 6 syllables, 4+4+4=12 syllables, and 7 syllables, respectively. Each line would end in a rhyming word, but the second line would have two words that rhyme in the middle.

The rhyme index is looked up in the rhyme dictionary for the song (```rhyme_dict```). This is not the *near rhyme dictionary* which is a list of all words known to nearly rhyme. The rhyme dictionary for the song is a list of words that the system should try to rhyme with when it gets to the end of a segment. If there is no entry in the dictionary, the system will generate forward unconstrained the appropriate number of syllables and the last word will be added to the ```rhyme_dict```. If the rhyme index is in the dictionary, then the system will pick a word that rhymes to end the segment in. If near rhyming is used, then near rhymes are added to the dictionary.In the ```rhyme_dict``` each rhyme index is associated with a list of words to rhyme with.

The number of syllables for a segment can be indicated with ```-1``` if the rhyme index for the segment has been replaced by a string. The ```-1``` tells the system to use however many syllables are in the string.

If the rhyme index for a segment is given as ```-1``` this tells the system to not try to rhyme with this segment.

A segment can take a third parameter, special instructions on how to handle the segment, ```(syllables, rhyme_index, command)```. Currently the only command is ```":end"``` which tells the system that this segement should end in a period. 

There is a post-processing step to add text to the beginning or ending of lines. It is useful for "ooh"s and other things that one might not want the neural language models to pick up on and include in the generation if they are given instead of rhyme indices. Post-processing is specified with a special dictionary (```post```) where each key is a number indicating the line number to be post-processed. The value is a tuple ```(pre, post)``` where ```pre``` is a string to add to the beginning of the line and ```post``` is a string to add to the end of the line. Either can be ```None```.


In [0]:
rhyme_dict = {}   # Rhyming lines are indicated by number. What word should certain lines rhyme with?
post = {}         # Post-processing scheme

# BEAT IT, MICHAEL JACKSON
scheme = [
#They told him don't you ever come around here
          (11, 1), #0
#Don't want to see your face, you better disappear
          [(6, -1), (6, 1)], #1
#The fire's in their eyes and their words are really clear
          [(5, -1), (7, 1)], #2
#So beat it, just beat it
          (3, 2, ':end'), #3
#You better run, you better do what you can
          (11, 3), #4
#Don't want to see no blood, don't be a macho man
          [(6, -1), (6, 3)], #5
#You want to be tough, better do what you can
          [(5, -1), (6, 3)], #6
#So beat it, but you want to be bad
          [(3, 2), (6, -1)], #7
#Just beat it, beat it, beat it, beat it
          (3, 2, ':end'), #8
#No one wants to be defeated
          (8, 2), #9
#Showin' how funky and strong is your fight
          (10, 4), #10
#It doesn't matter who's wrong or right
          (9, 4), #11
#Just beat it, beat it
          (3, 2), #12
#Just beat it, beat it
          (3, 2), #13
#Just beat it, beat it
          (3, 2), #14
#Just beat it, beat it
          (3, 2, ':end') #15
]
#rhyme_dict[1] = ['here']
post[3] = (None, ' just {repeat 2}.')
post[8] = (None, ' ({repeat 2}), {repeat 2}, ({repeat 2})')
post[12] = (None, ' ({repeat 2})')
post[13] = (None, ' ({repeat 2})')
post[14] = (None, ' ({repeat 2})')
post[15] = (None, ' ({repeat 2}), oooooh')

## Run!

Set the context to bias the agent toward a particular topic or theme. This will be added to the beginning of the lyrics as hidden text. 

In [0]:
context = "My favorite food is tacos" #@param {type:"string"}

**Run the generator.**

Set ```use_near_rhymes=True``` to use near rhymes or set to ```False``` to use perfect rhymes.

Set ```recontextualize=True``` to try to force the generator to attend to the original context more often. This causes the original context to be added after every sentence break to try to steer the generators back to the context. It won't show up in the final output.

Set ```interactive=True``` to manually select candidates for each segment.

In [0]:
RUN_ADVANCED_MODE = False #@param {type:"boolean"}
use_near_rhymes = True #@param {type:"boolean"}
recontextualize = False #@param {type:"boolean"}
interactive = False #@param {type:"boolean"}

# Run the generator
if RUN_ADVANCED_MODE:
  lines = run(context, scheme, rhyme_dict, 
              use_near_rhymes=use_near_rhymes, 
              post=post, 
              recontextualize=recontextualize, 
              interactive=interactive)
  # Print the outcome
  pretty_print(lines)
  # Save the phonentics cache
  save_phone_cache(PHONE_CACHE, PHONE_CACHE_PATH)


# Generate Karaoke Video

To do this you need an mp3 or mp4 file of music without vocals. A good way to do this is to find an existing karaoke video on YouTube. 

You will also need to know how many seconds each line of lyrics takes, down to at least the tenth of a second (but probably down to a millisecond to get the timing aligned).

## Install packages

In [34]:
!pip install pytube3

In [35]:
import os
from PIL import Image, ImageDraw, ImageFont
import moviepy.editor as mpe
from pytube import YouTube

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)1515520/45929032 bytes (3.3%)4497408/45929032 bytes (9.8%)8306688/45929032 bytes (18.1%)12189696/45929032 bytes (26.5%)16089088/45929032 bytes (35.0%)19767296/45929032 bytes (43.0%)23527424/45929032 bytes (51.2%)27312128/45929032 bytes (59.5%)31162368/45929032 bytes (67.8%)34938880/45929032 bytes (76.1%)37396480/45929032 bytes (81.4%)41222144/45929032 bytes (89.8%)4

## Video Creation

Need to set the font

In [0]:
# Generate a list of fonts available on the OS
#!fc-list
# Set the font
FONT = ImageFont.truetype("/usr/share/fonts/truetype/liberation/LiberationSans-Regular.ttf", size=24)

Helper functions

In [0]:
TEXT_OFFSET = (10, 10)
EPSILON = 0.01

def secs_to_millis(secs):
  return secs*1000

def convert(gif, mp4):
    os.system("ffmpeg -f gif -i " + gif + " " + mp4)

Download from YouTube

In [0]:
REMOVE_FROM_FILENAME = ['.', ',', '*']

def download_youtube(url):
  yt = YouTube(url)
  title = yt.title
  stream = yt.streams.filter(file_extension='mp4').first() #only_audio=True
  stream.download()
  return ''.join(c for c in title if not c in REMOVE_FROM_FILENAME) + '.mp4' 

Frame creation

In [0]:
 def create_frame(lines, highlight, fnt, background = "black", text_color = 'white', highlight_color = 'yellow', frame_wh = (1024, 768), text_offset_xy = (0, 0)):
    text_x, text_y = text_offset_xy
    spacing = frame_wh[1] / len(lines)
    img = Image.new('RGB', frame_wh, background)
    draw = ImageDraw.Draw(img)
    # draw.ellipse takes a 4-tuple (x0, y0, x1, y1) where (x0, y0) is the top-left bound of the box
    # and (x1, y1) is the lower-right bound of the box.
    for i, line in enumerate(lines):
      color = text_color if i != highlight else highlight_color
      draw.text((text_x, text_y + (i * spacing)), line, font = fnt, fill=color)
    return img

In [0]:
def make_video_by_durations(filename, audio_file_or_youtube, 
                            lines, start_time, durations, 
                            music_artist, music_title, attribution = 'Weird A.I. Yankovic', 
                            new_title = None,
                            frame_wh = (1024, 768),
                            for_twitter = False):
  # Copy durations
  durations = list(map(lambda x:secs_to_millis(x), [start_time] + durations))
  frames = []

  # Make a title if none given
  if new_title is None:
    init_line = lines[0].split()
    new_title = BLANK.join(init_line[0:min(4, len(init_line))])

  # Make title frame
  music_by = 'Music by: ' + music_artist + ' ("' + music_title + '")'
  lyrics_by = 'Lyrics by: ' + attribution
  title_lines = [new_title, music_by, lyrics_by]
  temp_file_gif = filename + '.TEMP_GIF.gif'
  temp_file_mp4 = filename + '.TEMP_VIDEO_MP4.mp4'
  title_frame = create_frame(title_lines, 0, FONT, frame_wh=frame_wh, text_offset_xy=TEXT_OFFSET)
  frames.append(title_frame)

  # If we can, get the lyrics up early
  if start_time > 5.0:
    title_duration = 5.0
    interstitial_duration = start_time - 5.0
    if start_time > 10.0:
      title_duration = 10.0
      interstitial_duration = start_time - 5.0
    durations = [secs_to_millis(title_duration), secs_to_millis(interstitial_duration)] + durations[1:]
    preview_frame = create_frame(lines, -1, FONT, frame_wh=frame_wh, text_offset_xy=TEXT_OFFSET)
    frames.append(preview_frame)

  # Make rest of frames
  for i, line in enumerate(lines):
      new_frame = create_frame(lines, i, FONT, frame_wh=frame_wh, text_offset_xy=TEXT_OFFSET)
      frames.append(new_frame)

  # End lyrics frame
  durations.append(secs_to_millis(EPSILON))
  end_frame = create_frame(lines, -1, FONT, frame_wh=frame_wh, text_offset_xy=TEXT_OFFSET)
  frames.append(end_frame)

  # Save into a GIF file 
  frames[0].save(temp_file_gif, format='GIF',
                append_images=frames[1:], save_all=True, duration=durations)
  # Convert to MP4
  convert(temp_file_gif, temp_file_mp4)
  # Load mp4
  my_clip = mpe.VideoFileClip(temp_file_mp4)
  # Load audio (mp3 or mp4)
  audio_file = audio_file_or_youtube
  if 'https://www.youtube.com' in audio_file_or_youtube or 'https://youtu' in audio_file_or_youtube:
    audio_file = download_youtube(audio_file_or_youtube)
  # Sometimes there are naming problems with the youtube download
  while not os.path.exists(audio_file):
    audio_file = input("File not found. Please enter the name of the file: ")
  # Get audio
  audio_clip = mpe.AudioFileClip(audio_file)
  # For Twitter
  if for_twitter:
    audio_clip = audio_clip.subclip(0, 120)
  # Add audio to video
  my_clip = my_clip.set_audio(audio_clip)
  # Write mp4
  my_clip.write_videofile(filename, 
                        audio=True,
                        codec='libx264', 
                        audio_codec='aac', 
                        temp_audiofile='temp_audio.m4a', ) # default codec: 'libx264', 24 fps
  # Clean up
  os.remove(temp_file_gif)
  os.remove(temp_file_mp4)

def make_video_by_line_timing(filename, audio_file_or_youtube, 
                              lines, timing, total_duration, 
                              music_artist, music_title, attribution = 'Weird A.I. Yankovic',
                              new_title = None, 
                              frame_wh = (1024, 768),
                              for_twitter = False):
  counter = timing[0]
  durations = [counter]
  for time in timing[1:]:
    dur = time - counter
    durations.append(dur)
    counter = time
  durations.append(total_duration - counter)
  make_video_by_durations(filename, audio_file_or_youtube, lines, durations[0], durations[1:], music_artist, music_title, attribution, new_title, frame_wh, for_twitter)

## Run the Generator

There are two ways to make the clip. 

1. If you know the duration of each line of lyrics, use ```make_video_by_durations()```. You will also need to set the start time of the first line so that lyrics match the music.

2. If you know the start time of each line, use ```make_video_by_line_timing()```. You will also need to know the total duration of the lyrics.

If you want a video that you can post to Twitter, it must be less than 2 minutes; setting ```for_twitter=True``` will clip the video. 

In [0]:
# BEAT IT, MICHAEL JACKSON
# Timing works for https://www.youtube.com/watch?v=cuSi8yR9rKk
start_time = 43.2
line_durations = [3.0, 3.3, 3.4, 4.2, 2.8, 3.6, 3.3, 3.5, 3.7, 3.5, 3.4, 3.3, 1.9, 1.9, 1.4, 2.3]

To run the karaoke video generation, set ```MAKE_KARAOKE_VIDEO=True``` and fill in the required values using the form to the right.

In [0]:
MAKE_KARAOKE_VIDEO = False #@param{type:"boolean"}
save_filename = 'my_karaoke.mp4' #@param {type:"string"}
title = 'Sassafras' #@param {type:"string"}
audio_file_or_youtube = 'https://www.youtube.com/watch?v=cuSi8yR9rKk' #@param {type:"string"}
music_artist = 'Michael Jackson' #@param {type:"string"}
original_music_title = 'Beat It' #@param {type:"string"}
attribution = 'Weird A.I. Yankovic and its user' #@param {type:"string"}
for_twitter = True #@param {type:"boolean"}

if MAKE_KARAOKE_VIDEO: 
  make_video_by_durations(save_filename, audio_file_or_youtube, 
                          lines, start_time, line_durations, 
                          music_artist, original_music_title, 
                          new_title=title,
                          attribution=attribution,
                          for_twitter=for_twitter) 

# License

Copyright 2020 Mark Owen Riedl

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.